# Start with your Retrieval Pipeline

A common problem is that end-to-end testing with language models is slow, expensive and hard to scale. It's extremely common to focus on evaluating the quality of generated content but this tends to be hard to scale and expensive to implement. 

This is where retrieval-focused evaluation shines. By focusing specifically on retrieval, we can test faster, iterate quicker and measure objectively. This is because retrieval metrics such as precision and recall are significantly faster and cheaper to compute. Most importantly, with a solid retrieval pipeline, we can start supporting several initial use cases to start gathering feedback on.

 
| Aspect | Generation evals | Retrieval evals |
|--------|------------------|-----------------|
| Testing for | Team lead: "Looks better now" (based on metric evaluations that don't capture usage behavior) | Recall, precision@K |
| Speed of tests | Slow (~1s to 10s per test) | Fast (~10ms to 800ms per test) |
| Cost of tests | $100s per run | Negligible per run |
| Frequency of tests | Infrequent | Frequent |
| Speed of test iterations | Slow iterations (tests that could take minutes to hours) | Fast iterations |
| Ease of test scalability | Difficult | Easy |

## Synthetic Questions

Since we may not have access to real user queries initially, generating synthetic questions allows us to simulate potential user interactions with our system. This approach helps us establish a performance baseline for our retrieval pipeline and identify any retrieval issues early on, ensuring that our system is robust before deployment. 

This is a never-ending process. Once you do get user traffic, you'll want to keep generating more synthetic questions and start blending in user queries to make sure your system's retrieval is still doing well.



# Improving Text-2-SQL with Synthetic Questions

A common use case for Language Models is to answer questions with SQL using Text-2-SQL models. These take in a user query and output a SQL query which can be used to retrieve the relevant information. 

```
Text : Hey could you help me find the top 5 most popular items in our store?
Query: SELECT item_name, COUNT(*) as popularity FROM items GROUP BY item_name ORDER BY popularity DESC LIMIT 5
```

It's common to approach this as a generation task where the solution is to keep iterating on the prompt until the model outputs the correct SQL query. Generation is difficult to evaluate since we need to manually check the output. Automating this is a challenging task since a valid SQL query can be written in many different ways.

Instead, it's much easier in this case to start by looking at the retrieval step. Since most companies have complex business use cases or might be using proprietary data, it's difficult to know what the final query should look like. That's where retrieval comes in.

We can think of this as a retrieval task where we're trying to find the most relevant SQL query for a given user query. To do so, we can take each individual SQL query and generate a synthetic question for it. 

## Case Study : Bird-Bench

For this case-study, we'll be using the Bird-Bench dataset. This is a huge Text-2-SQL dataset which originally contained a collection of text questions to a corresponding sql query. We'll be using the dev split of this dataset for this case study that provides ~1500+ sql snippets that involves ~95 different tables that we can use. 

Each example in our dataset contains three things

- `id` : This is a unique identifier for each query
- `query` : This is a sample SQL query 
- `difficulty` : This is a label that indicates how difficult the query is to generate. It can be either `simple`, `moderate` or `challenging`. For this case study, we'll only be using the `challenging` queries so that we can generate more difficult questions.

Let's download our dataset and take a look at the first example

In [1]:
import datasets

dataset = datasets.load_dataset("567-labs/bird-rag")["train"]
dataset[0]

{'id': '0',
 'query': "SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1",
 'difficulty': 'simple'}

Before we generate questions, let's take a look at what a synthetic question might look like by looking at our query below.

> SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1


We can see that this query is doing a few things

1. First it's looking at the percentage of students that are recieving free meals 
2. It's restricting this to only schools in Alameda County
3. Finally, it's ordering the results by this percentage in descending order and limiting the results to the top 1

In short, it's trying to find the school in Alameda County with the highest percentage of students recieving free meals. So, what would be a potential question that we could ask which this query snippet would be highly relevant for?

Well, a potential question could be

- What is the school with the highest percentage of students recieving free meals in Alameda County?
- How do we find the number of students recieving free meals in Seattle?

These are queries which this snippet would be highly relevant for. Our goal with synthetic questions are to create a diverse set of questions here which would be able to test how consistently we retrieve the relevant snippet for a user query.

## Generating Synthetic Questions

Now that we understand what we're trying to do, let's see how we can load in our data and start generating synthetic questions. We'll do so in 3 steps

1. Define our Pydantic models for our data
2. Load in our data 
3. Start generating questions

It's useful to start by defining models that explicitly represent the data that we want to work with. While this might seem like overkill for a small dataset, it ensures that our data is in the right format as we start generating our synthetic questions. 

For this specific example, we'll have 3 different pydantic models

1. `Chunk` : This represents how we're representing our data from the dataset
2. `Question` : This is the synthetic question that we'll be getting our model to generate
3. `ChunkEval` : This represents a single question-chunk pair that we'll be uploading to Braintrust as a dataset later on to be used for benchmarking in `benchmark_retrieval.py`


In [2]:
from pydantic import BaseModel


class Chunk(BaseModel):
    chunk_id: str
    text: str


class Question(BaseModel):
    chain_of_thought: str
    question: str


class ChunkEval(BaseModel):
    chunk_id: str
    question: str
    chunk: str

### Using Structured Outputs to Generate Questions

Now let's see how we can generate a question for a given SQL snippet. 

Remember that we want to generate a question that should either be answerable by the data returned by the SQL snippet or a snippet that is similar to the SQL snippet. To do so, we'll be using `instructor` to handle our generation. This is mainly due to two reasons

1. We get to pass in a Pydantic object as a response model which helps with error handling and ensures that the output is in the right format
2. With `jinja` templating, we can easily format our prompt by using `jinja` variables and using the `context` object to pass in the relevant data.

You can definitely use the default OpenAI client but it's much simpler to just use `instructor`.

In [3]:
import openai
import instructor
from asyncio import Semaphore, timeout
from tqdm.asyncio import tqdm_asyncio as asyncio
from tenacity import retry, stop_after_attempt, wait_fixed

client = instructor.from_openai(openai.OpenAI())

sql_snippet = "SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1"

resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": """
        Generate a question and answer pair that the following SQL snippet below will be able to answer. 

        The question should 
        - Be answerable only by the data that will be returned by the SQL snippet
        - Not mention specific information in the SQL snippet directly
        
        
        SQL Snippet:
        {{ snippet }}
        """,
        }
    ],
    response_model=Question,
    context={"snippet": sql_snippet},
)

print(resp.question)


What is the highest proportion of free meals provided to the enrolled K-12 students in any school district within Alameda County?


The generated question is great, it's not only answerable by the data returned by the SQL snippet but it also doesn't mention specific information in the SQL snippet directly. Instead, it references the end result of why we would want the data - to make some conclusion about why we would want to find the school with the highest percentage of students recieving free meals in Alameda County.

When generating a large amount of data at a single go, it's useful here to keep two things in mind

1. We want to stay within our rate limits
2. We need to make sure we're generating data in the form that we want

It's useful therefore to always generate a few small samples, ensure that the output is in the right format and then scale up from there. Our questions seem to be in the right format so let's scale up. To do so, we'll use the `asyncio` library to run our requests in parallel, a semaphore to limit the number of concurrent requests and the async OpenAI client to handle our requests.


In [9]:
client = instructor.from_openai(openai.AsyncOpenAI())


@retry(stop=stop_after_attempt(3), wait=wait_fixed(10))
async def generate_questions(chunk: Chunk, sem: Semaphore) -> ChunkEval:
    async with sem, timeout(30):
        resp = await client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": """
                Generate a question based off the following SQL snippet. Make sure that it's a question that can only be answered by the SQL snippet.

                
                Make the question more challenging by doing one of the following.
                - Modifying the time period that we're filtering on (Eg. if the snippet has filtered by the entire year, look at 6 months or 1.5 years)
                - Changing the value of the filter (Eg. if the snippet is looking at the results in Canada, change the question to ask about Toyko or Mexico instead)
                - Adding in some useless information: Eg. Add information about the weather, a random event or a backstory that isn't mentioned in the snippet.

                The final question should be at most 2 sentences long.

                SQL Snippet:
                {{ snippet }}
                """,
                }
            ],
            response_model=Question,
            context={"snippet": chunk.text},
            timeout=20,
        )

        return ChunkEval(
            chunk_id=chunk.chunk_id,
            question=resp.question,
            chunk=chunk.text,
        )


sem = Semaphore(10)
dataset = [
    item
    for item in datasets.load_dataset("567-labs/bird-rag")["train"]
    if item["difficulty"] == "challenging"
]
dataset = [Chunk(chunk_id=item["id"], text=item["query"]) for item in dataset]

coros = [generate_questions(chunk, sem) for chunk in dataset]
questions: list[ChunkEval] = await asyncio.gather(*coros)

100%|██████████| 145/145 [00:46<00:00,  3.13it/s]


Now that we've generated our questions, let's take a look at what they look like.

In [10]:
from rich import print

questions[20]
print(questions[20].question)
print(questions[20].chunk)


During a scientific conference in Tokyo, researchers are exploring new compounds, specifically those related to 
molecule_id 'TR120'. What distinct elements can be identified from this molecule?

SELECT DISTINCT T1.element, T2.label FROM atom AS T1 INNER JOIN molecule AS T2 ON T1.molecule_id = T2.molecule_id 
WHERE T2.molecule_id = 'TR060'

We can see that for each individual snippet, we generated a question that directly requires the SQL snippet's for data or for reference to be answered. This is crucial in ensuring that we're able to evaluate our retrieval pipeline well.

## Uploading our Dataset

> Braintrust is a end to end platform for building LLM applications. We'll be using it in `benchmark_retrieval.ipynb` to also evaluate our questions

When running eval datasets, you need to start thinking about where to store them. We recommend using braintrust for this, it simplifies a significant amount of the process and makes it easier to keep track of all your experiments/production logs.

Uploading a dataset is as simple as calling `braintrust.init_dataset` with the project and name of the dataset that we want to create. Note here that we're passing in the `chunk_id` as the `id` for each question. This helps us to avoid uploading duplicate questions when creating our dataset.

You can verify that this is the case by running the cell a few times and seeing that the total number of questions stays the same.

In [11]:
import braintrust


dataset = braintrust.init_dataset(project="Text-2-SQL", name="Bird-Bench-Questions")
for question in questions:
    dataset.insert(
        id=question.chunk_id,
        input=question.question,
        expected=[question.chunk],
        metadata={"chunk_id": question.chunk_id, "chunk": question.chunk},
    )

print(dataset.summarize())

DatasetSummary(
    project_name='Text-2-SQL',
    dataset_name='Bird-Bench-Questions',
    project_url='https://www.braintrust.dev/app/567/p/Text-2-SQL',
    dataset_url='https://www.braintrust.dev/app/567/p/Text-2-SQL/datasets/Bird-Bench-Questions',
    data_summary=DataSummary(new_records=145, total_records=145)
)